In [ ]:
#| default_exp fastmcp_tools
#| test: false

#"""Shared FastMCP SSE server exposing parking & route tools."""


In [ ]:
#| export
#| eval: false

import logging, itertools, json, re
from functools import lru_cache
from dataclasses import dataclass, field
from typing import List, Dict, Any
from DataTalks.api_clients import Mode as Route_Mode
import httpx, yaml
from fastmcp import FastMCP  


from DataTalks.parking_tools import (
    nearest_parking_ids,
    parking_status,
    facility_prediction,
)

from DataTalks.api_clients import (
    Point,
    osrm_car,
    osrm_walk,
    osrm_bike,
    digitransit_pt,
    geocode, Route
)


ImportError: cannot import name 'osrm_bike' from 'DataTalks.api_clients' (/home/nata/DataTalks/DataTalks/api_clients.py)

In [ ]:
#| export
#| eval: false

mcp = FastMCP(name="tools")
app = mcp.http_app(transport="sse", path="/sse")

In [ ]:
#| export
#| eval: false

INDEX_URL = "https://api.apis.guru/v2/list.json"

In [ ]:
#| export
#| eval: false

import re, httpx, yaml, json


VALID = re.compile(r"[^a-zA-Z0-9_-]")

def load_and_clean_spec(url: str) -> dict:
    text = httpx.get(url, timeout=30).text
    spec = yaml.safe_load(text) if url.endswith((".yml", ".yaml")) else json.loads(text)
    for path, methods in spec["paths"].items():
        for verb, op in methods.items():
            # ensure an operationId exists
            op_id = op.get("operationId") or f"{verb}_{path.strip('/').replace('/','_')}"
            op_id = VALID.sub("_", op_id)[:64]           # 🔑  clean + truncate
            op["operationId"] = op_id.lower()
    return spec                         


In [ ]:
#| export
#| eval: false


_VALID_RE   = re.compile(r"^[a-zA-Z0-9_-]{1,64}$")
_SANITIZE_RE = re.compile(r"[^a-zA-Z0-9_-]")

def _clean(name: str) -> str:
    """
    Return a schema-compliant tool name (letters, digits, _-, ≤64 chars).
    Dots/slashes/spaces → “_”; consecutive underscores collapsed.
    """
    cleaned = _SANITIZE_RE.sub("_", name)           # illegal → _
    cleaned = re.sub(r"__+", "_", cleaned)          # collapse runs
    return cleaned[:64] or "t"                      # never empty

def _dedupe(names: set[str], base: str) -> str:
    """If *base* already exists, append '_1', '_2' etc. until unique."""
    if base not in names:
        return base
    for i in itertools.count(1):
        candidate = f"{base}_{i}"
        if candidate not in names and len(candidate) <= 64:
            return candidate


In [ ]:
#| export
#| eval: false


SPEC_CACHE: Dict[str, dict] = {}

@lru_cache
def _get_index() -> Dict[str, dict]:
    return httpx.get(INDEX_URL, timeout=10).json()

def _score(title: str, query: str) -> float:
    q, t = query.lower(), title.lower()
    return (q in t) * 5 + sum(w in t for w in q.split())

def _pick_spec_url(ver: dict) -> str | None:
    """Return the best downloadable spec URL or None if absent."""
    return (
        ver.get("openapiUrl")      # OpenAPI 3+
        or ver.get("swaggerUrl")   # Swagger 2.0
        or ver.get("link")         # catch-all (rare)
    )

def _requires_credentials(spec: dict) -> bool:
    """Return *True* if the OpenAPI spec declares any mandatory security.

    Logic (conservative – errs on the side of *requiring* auth):
    • If *components.securitySchemes* is **missing** → assume *no* creds.
    • Else, if the global *security* list is **present & non‑empty* → creds.
    • Else, scan every operation for a non‑empty *security* list → creds.

    We don’t attempt to be clever about *optional* auth – that’s rarely used
    in public APIs and would require heuristic scoring beyond this scope.
    """

    if not spec.get("components", {}).get("securitySchemes"):
        return False 

    # Global requirement?
    if spec.get("security"):
        return True

    # Operation‑level requirement?
    for path_item in spec.get("paths", {}).values():
        for verb, operation in path_item.items():
            if isinstance(operation, dict) and operation.get("security"):
                return True

    return False 



def search_public_apis(query: str, limit: int = 10) -> List[Dict]:
    """Return at most *limit* APIs whose specs **don’t** require creds."""

    idx = _get_index()
    # Rough relevance score identical to the original helper
    def __score(title: str, q: str) -> float:
        ql, tl = q.lower(), title.lower()
        return (ql in tl) * 5 + sum(w in tl for w in ql.split())

    ranked: List[tuple[str, str, str]] = sorted(
        (
            (api_id, ver_meta["info"]["title"], _pick_spec_url(ver_meta))
            for api_id, meta in idx.items()
            if (ver_meta := meta["versions"][meta["preferred"]])  # := 3.8+
        ),
        key=lambda t: __score(t[1], query),
        reverse=True,
    )

    # Keep only specs that have a downloadable URL *and* require no auth
    results: List[Dict] = []
    for api_id, title, spec_url in ranked:
        if not spec_url:
            continue
        try:
            spec = httpx.get(spec_url, timeout=8).json()  # small JSON files
        except Exception:
            continue  # skip broken links silently
        if _requires_credentials(spec):
            continue
        results.append({"id": api_id, "title": title, "spec_url": spec_url})
        if len(results) >= limit:
            break
    return results


async def _load_openapi_spec(url: str) -> dict:
    if url in SPEC_CACHE:
        return SPEC_CACHE[url]
    async with httpx.AsyncClient(timeout=10) as cx:
        res = await cx.get(url)
    res.raise_for_status()
    SPEC_CACHE[url] = yaml.safe_load(res.text)
    return SPEC_CACHE[url]


# fastmcp_tools.py  ────────────────────────────────────────────────────────
from pydantic import ValidationError 


@mcp.tool(
    name="mount_openapi",
    description=(
        "Mount an OpenAPI or Swagger spec so its operations become callable "
        "tools. Args: name, spec_url, base_url (optional), http_timeout (s)."
    ),
)
async def mount_openapi(
    *,
    name: str,
    spec_url: str,
    base_url: str | None = None,
    http_timeout: float = 10.0,
    mcp_root: object | None = None,      # injected by FastMCP, may be omitted
) -> List[str]:
    """
    Downloads `spec_url`, validates it, converts every operation into a
    FastMCP tool and mounts the sub-server under `/{name}`.

    Returns the list of newly created **tool names**.
    Raises ToolError if the spec is invalid or an upstream URL cannot be
    inferred – the calling agent can catch this and try another API.
    """
    # ── 1. download & validate spec ────────────────────────────────────
    try:
        spec = await _load_openapi_spec(spec_url)          # your cached helper
    except Exception as e:
        raise Exception(f"Could not fetch spec ({e})")

    # ── 2. ensure every op has operationId ─────────────────────────────
    HTTP_VERBS = {"get", "put", "post", "delete", "options",
                  "head", "patch", "trace"}
    for path, methods in spec.get("paths", {}).items():
        slug = path.strip("/").replace("/", "_")
        for verb, op in methods.items():
            if verb.lower() not in HTTP_VERBS or not isinstance(op, dict):
                continue
            op.setdefault("operationId", f"{verb}_{slug}")

    # ── 3. determine upstream base-URL ─────────────────────────────────
    def _infer_upstream_from_swagger2(sw: dict) -> str | None:
        host = sw.get("host")
        if not host:
            return None
        scheme = (sw.get("schemes") or ["https"])[0]
        return f"{scheme}://{host}{sw.get('basePath', '')}"

    upstream = (
        base_url
        or spec.get("servers", [{}])[0].get("url")           # OpenAPI 3.*
        or _infer_upstream_from_swagger2(spec)               # Swagger 2.0
    )
    if not upstream:
        raise Exception("Cannot determine upstream URL (no servers / host).")
    
    spec = load_and_clean_spec(spec)

    # ── 4. build HTTP client & sub-server ─────────────────────────────
    client = httpx.AsyncClient(base_url=upstream, timeout=http_timeout)
    try:
        sub = FastMCP.from_openapi(
            openapi_spec=spec,
            client=client,
            name=name,
            all_routes_as_tools=True,
        )
    except Exception as e:
        raise Exception(f"Failed to build FastMCP server: {e}") from e

    # flag for UI (does the spec declare any securitySchemes?)
    sub.requires_auth = bool(spec.get("components", {}).get("securitySchemes"))
    
    # ── 5. mount on the root router and return tool names ─────────────
    (mcp_root or mcp).mount(name, sub)
    
    # ── 5.b ensure resulting tool names obey OpenAI pattern ───────────
    valid_names: set[str] = {t.name for t in (await sub.get_tools()).values()}
    for t in (await sub.get_tools()).values():
        if _VALID_RE.match(t.name):
            continue
        new_name = _dedupe(valid_names, _clean(t.name))
        t.name = new_name
        valid_names.add(new_name)
    
    
    
    return [t.name for t in (await sub.get_tools()).values()]

    


@mcp.tool(
    name="quick_mount_openapi",
    description=(
        "Search the public-API index by keyword and mount the first spec that "
        "doesn’t require credentials."
    ),
)
async def quick_mount_openapi(
    query: str,
    mcp_root: object | None = None,   # optional override for tests / sub-routers
) -> List[str]:
    """
    1. Search the APIs-Guru index for *public* specs (no security schemes)
       matching **query**.
    2. Try them one by one until `mount_openapi` succeeds; return the list of
       newly created tool names.
    3. If none mount successfully, return `[]` **and** stash a list of
       candidate APIs that *do* require creds in
       `mcp_root.state["needs_creds"]` so the UI can prompt the user.
    """

    # 1️⃣  Try at most 10 public hits
    for hit in search_public_apis(query, limit=10):
        try:
            slug = re.sub(r"[^a-z0-9_]+", "_", hit["id"].lower())[:30]
            return await mount_openapi(          # ← async + await
                name=slug,
                spec_url=hit["spec_url"],
                mcp_root=mcp_root,
            )
        except Exception:
            continue          # keep trying next candidate

    # 2️⃣  Nothing mounted → gather *all* candidates (public or not)
    candidates: List[Dict[str, Any]] = []
    for api_id, meta in _get_index().items():
        ver_meta = meta["versions"][meta["preferred"]]
        spec_url = _pick_spec_url(ver_meta)
        if not spec_url:
            continue
        candidates.append({
            "id": api_id,
            "title": ver_meta["info"]["title"],
            "spec_url": spec_url,
            "needs_credentials": True,   # we already tried the public ones
        })

    if mcp_root is not None:
        # ensure a consistent place for the UI/agent to inspect
        mcp_root.state["needs_creds"] = candidates

    return []   # signal to the agent/UI that mounting failed


In [ ]:
#| export
#| eval: false

@app.on_event("startup")
async def _load_specs() -> None:
    """Warm the APIs.guru index and log a preview (first 10 records)."""
    index: dict = _get_index()          # synchronous helper you already wrote


In [ ]:
#| export
#| eval: false


@mcp.tool()
async def car_route(
    src_lat: float, src_lon: float,
    dst_lat: float, dst_lon: float,
) -> Route:
    "Car route via public OSRM demo."
    return await osrm_car(Point(lat=src_lat, lon=src_lon),
                          Point(lat=dst_lat, lon=dst_lon))


@mcp.tool()
async def walk_route(
    src_lat: float, src_lon: float,
    dst_lat: float, dst_lon: float,
) -> Route:
    "Walking route via OSRM."
    return await osrm_walk(Point(lat=src_lat, lon=src_lon),
                           Point(lat=dst_lat, lon=dst_lon))


@mcp.tool()
async def bike_route(
    src_lat: float, src_lon: float,
    dst_lat: float, dst_lon: float,
) -> Route:
    "Walking route via OSRM."
    return await osrm_bike(Point(lat=src_lat, lon=src_lon),
                           Point(lat=dst_lat, lon=dst_lon))

@mcp.tool()
async def public_transport_route(
    src_lat: float, src_lon: float,
    dst_lat: float, dst_lon: float,
    dataset: str = "hsl",
) -> Route:
    """Fastest public-transport itinerary via Digitransit Routing v2."""
    return await digitransit_pt(Point(lat=src_lat, lon=src_lon),
                                Point(lat=dst_lat, lon=dst_lon),
                                dataset=dataset)

@mcp.tool()
async def geocode_osm(query: str) -> dict:
    """First OpenStreetMap location match for *query*."""
    try:
        return (await geocode(query)).model_dump()
    except LookupError as e:
        # surface a predictable ToolError so the agent can fall back
        raise ValueError(str(e))


In [ ]:
#| export
#| eval: false

@mcp.tool()
async def nearest_parking(
    lat: float, lon: float, radius_m: int = 800
) -> dict[str, int]:
    """
    Return `{capacityType: facility_id}` for the closest facilities
    within *radius_m* metres of (*lat*, *lon*).
    """
    return await nearest_parking_ids(lat, lon, radius_m=radius_m)


@mcp.tool()
async def urheilupuisto_parking_status(
    facility_id: int = 285,                  # Urheilupuisto default
    capacity_type: str = "CAR",
    usage: str = "PARK_AND_RIDE",
) -> dict:
    """
    **Parking only.** Returns real-time utilisation of a *Fintraffic* facility.
    Not related to weather or other live data.
    """
    return await parking_status(
        facility_id,
        capacity_type=capacity_type,
        usage=usage,
    )


@mcp.tool()
async def urheilupuisto_parking_forecast(
    facility_id: int = 285,
    hours: int = 24,
) -> list[dict]:
    """
    Fintraffic capacity forecast for *facility_id* over the next *hours*.
    """
    return await facility_prediction(facility_id, hours=hours)


In [ ]:
#| export
#| eval: false

@dataclass
class SegmentEmission:
    mode: Route_Mode
    distance_m: float
    co2e_g: float

@dataclass
class TripEmissionEstimate:
    segments: List[SegmentEmission]
    total_distance_m: float
    total_co2e_g: float

_EMISSION_FACTORS: dict[str, float] = {
    "car" : 171.0,
    "bike": 21.0,
    "walk": 56.0,
    "pt"  :  65.0,   # typical urban-public-transport mix (bus+rail)
}

@mcp.tool(
    name="estimate_trip_emissions",
    description=(
        "Given a list of Route objects that form a multimodal trip, return "
        "segment-by-segment and total CO₂-e estimates using average factors."
    ),
)
def estimate_trip_emissions(routes: List[Route]) -> TripEmissionEstimate:
    segs: List[SegmentEmission] = []
    for rt in routes:
        factor = _EMISSION_FACTORS.get(rt.mode, 0.0) 
        co2_g  = factor * (rt.distance_m / 1000)
        segs.append(SegmentEmission(rt.mode, rt.distance_m, co2_g))

    total_d = sum(s.distance_m for s in segs)
    total_c = sum(s.co2e_g     for s in segs)

    return TripEmissionEstimate(
        segments=segs,
        total_distance_m=total_d,
        total_co2e_g=total_c,
    )
